In [13]:
# !pip install requests

In [14]:
# !pip install beautifulsoup4

In [15]:
# !pip install tqdm

In [16]:
# !pip install pandas

# 문제점

1. 암호화폐 뉴스 사이트들 대부분이 서로 크롤링 하는 경우가 많아서 이미 해당 사건으로부터 몇 십분에서 몇 시간, 며칠이 지난 후에 뉴스가 올라오는 경우가 되게 많음

예: A사이트는 B사이트를 크롤링해서 뉴스 게재 -> B사이트는 C사이트를 크롤링해서 뉴스 게재 -> C사이트는 A사이트를 크롤링해서 게재

2. 원인 불명의 문제로 인해 크롤링할 때 일부 뉴스 데이터들이 누락

# 라이브러리

In [8]:
# 파이썬 표준 라이브러리
import os
import json
import re
import random
import time
import traceback
from datetime import datetime
from functools import partial
from concurrent import futures

# 파이썬 서드파티 라이브러리
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 함수

In [ ]:
def request_html(
                url: str, headers: dict[str, str], allow_redirects: bool, timeout: int,
                 max_retry: int
                 ) -> str | None:
    """requests로 HTML 문서 정보를 불러오는 함수

    Args:
        url: 뉴스 URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
    
    Return:
        텍스트화한 HTML 문서 정보, str

        or 
    
        None
    """

    html = None

    for _ in range(max_retry):
        # requests로 HTML GET
        response = requests.get(url, headers=headers, allow_redirects=allow_redirects, timeout=timeout)
        # HTML 문서 정보를 불러오는 것에 성공하면 for문 중단
        if response.status_code == requests.codes.ok:
            html = response.text
            break
        # 실패한 응답 요청을 출력
        try:
            response.raise_for_status()
        except Exception:
            print()
            print(f'HTML 문서 정보 가져오기를 실패한 뉴스 URL : {url}')
            print(traceback.format_exc())
        time.sleep(random.uniform(0.35, 1.25))
    
    return html

In [2]:
def investing(
            news_url: str, headers: dict[str, str], allow_redirects: bool, timeout: int, max_retry: int,
              p0: re.Pattern, p1: re.Pattern, p2: re.Pattern, p3: re.Pattern
              ) -> dict[str, str, None] | None:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url_tag: 뉴스 URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
        p0: "입력: \r\n 2024- 12- 07- 오후 08:45"와같은 텍스트에서 ' \r\n '를 기준으로 분리하는 패턴
        p1: 디셉터에서 읽기 / Provided COINNESS / 이승훈 기자 123@gmail.com 등의 불필요한 텍스트 패턴
        p2: "발표했습니다.그리고"와 같이 마침표 다음에 유니코드 문자열이 바로 붙어있는 텍스트 패턴
        p3: "\xa0"가 1개 이상 연달아 붙어있는 텍스트 패턴

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str | None
            "author": 뉴스 작성자, str | None
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
            "note": 비고, str | None
        }

        or

        None
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # requests로 HTML GET
    html = request_html(url=news_url, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
    # HTML 문서 정보를 불러오는 것에 실패하면 None 반환
    if html is None:
        return None
    # BeautifulSoup로 parser
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', id='articleTitle')
    if title is None:
        title = None
    else:
        title = title.text.strip(' \t\n\r\f\v')

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
    if div:
        span = div[1].find('span')
        if span is None:
            first_upload_time = None
        else:
            span = span.text
            first_upload_time_list = p0.split(span)[1].split()
            y_m_d = '-'.join(times[:-1] for times in first_upload_time_list[:3])
            if first_upload_time_list[3] == '오전':
                ap = 'AM'
            else:
                ap = 'PM'

            first_upload_time = y_m_d + ' ' + ap + ' ' + first_upload_time_list[4]
        last_upload_time = None
    else:
        first_upload_time = None
        last_upload_time = None

    # 3. 뉴스 데이터의 기사 작성자
    author = None

    # 4. 뉴스 데이터의 본문
    article = soup.find('div', id='article')
    if article is None:
        content = ''
    else:
        while article.find("td") is not None:
            article.find("td").decompose()
        news_content = article.get_text(separator="\n", strip=True).split('\n')
        # 뉴스 데이터 본문의 데이터 전처리1
        # 디셉터에서 읽기 / Provided COINNESS / 이승훈 기자 123@gmail.com 등의 불필요한 텍스트 제거
        while news_content:
            if  p1.search(string=news_content[-1]) is None:
                break
            del news_content[-1]
            
        # 뉴스 데이터 본문의 데이터 전처리2
        # - 하나의 텍스트로 결합한 다음, 맨 앞/뒤의 화이트스페이스(whitespace) 문자 제거
        # - "발표했습니다.그리고"와 같이 마침표 다음에 유니코드 문자열이 바로 붙어있는 경우, 띄어쓰기 보정
        # - "\xa0"를 " "로 변경
        content = '\n\n'.join(news_content)
        content = p2.sub(repl=r'\g<1>. \g<2>', string=content)
        content = p3.sub(repl=' ', string=content)
        content = content.strip(' \t\n\r\f\v')

    # 5. 뉴스 웹사이트 이름
    website = 'Investing'

    # 6. 비고
    note = None

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = content
    info['news_url'] = news_url
    info['news_website'] = website
    info['note'] = note

    return info

In [3]:
def hankyung(
            news_url: str, headers: dict[str, str], allow_redirects: bool, timeout: int, max_retry: int,
              p0: re.Pattern
              ) -> dict[str, str, None] | None:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url_tag: 뉴스 URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
        p0: 인도 벵갈루루=양한나 블루밍비트 기자 sheep@bloomingbit.io / (사진=연합뉴스)와같은 텍스트 패턴

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str | None
            "author": 뉴스 작성자, str | None
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
            "note": 비고, str | None
        }

        or

        None
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # requests로 HTML GET
    html = request_html(url=news_url, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
    # HTML 문서 정보를 불러오는 것에 실패하면 None 반환
    if html is None:
        return None
    # BeautifulSoup로 parser
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', {"class": "headline"})
    if title is None:
        title = None
    else:
        title = title.text.strip(' \t\n\r\f\v')

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    upload_times = soup.find_all('span', {"class": "txt-date"})
    if upload_times:
        first_upload_time = upload_times[0].text
        first_upload_time = datetime.strptime(first_upload_time, '%Y.%m.%d %H:%M')
        first_upload_time = datetime.strftime(first_upload_time, '%Y-%m-%d %p %I:%M')
        last_upload_time = upload_times[1].text
        last_upload_time = datetime.strptime(last_upload_time, '%Y.%m.%d %H:%M')
        last_upload_time = datetime.strftime(last_upload_time, '%Y-%m-%d %p %I:%M')
    else:
        first_upload_time = None
        last_upload_time = None

    # 3. 뉴스 데이터의 기사 작성자
    author_list = soup.find_all('div', {"class": "author link subs_author_list"})
    if author_list:
        author_list = map(lambda x: x.find("a").text, author_list)
        author = ', '.join(author_list)
    else:
        author = None

    # 4. 뉴스 데이터의 본문
    articletxt = soup.find("div", id="articletxt")
    while articletxt.find("strong") is not None:
        articletxt.find("strong").decompose()
    while articletxt.find("figcaption") is not None:
        articletxt.find("figcaption").decompose()
    text_list = articletxt.get_text(separator="\n", strip=True).split('\n')
    while text_list:
        if p0.search(string=text_list[-1]) is None:
            break
        del text_list[-1]
        
    content = '\n\n'.join(text_list).strip(' \t\n\r\f\v')

    # 5. 뉴스 웹사이트 이름
    website = 'Hankyung'

    # 6. 비고
    note = None

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = content
    info['news_url'] = news_url
    info['news_website'] = website
    info['note'] = note

    return info

In [4]:
def crawling(website: str, news_websites_dict: dict[str, str]):
    """Crawling을 하는 함수

    Args:
        website: 웹사이트 이름, str
        news_websites_dict: 웹사이트 Dictionary, dict[str, str]

    Returns:
        pass
    """

    assert website in news_websites_dict, f'{website} is not target website.'

    match website:
        case 'Investing':
            pass

        case 'Hankyung':
            pass

        case 'Bloomingbit':
            pass

        case 'Coinreaders':
            pass

# 전역 변수 및 환경 설정

In [9]:
# cpu 갯수
workers = os.cpu_count()
print(workers)

8


In [10]:
news_websites_dict = {
                        'Investing': 'https://kr.investing.com/news/cryptocurrency-news',
                        'Hankyung': 'https://www.hankyung.com/koreamarket/news/crypto',
                        'Bloomingbit': 'https://bloomingbit.io/feed',
                        'Coinreaders': 'https://www.coinreaders.com/'
                      }

In [11]:
# User-Agent 변경을 위한 옵션 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': user_agent}

# requests 파라미터
allow_redirects = True # 리다이렉트 허용 여부
timeout = 150 # 응답 대기 허용 시간
max_retry = 10 # HTML 문서 요청 최대 재시도 횟수

# 경로
investing_path = r'.\Data\Investing_Data.json'
hankyung_path = r'.\Data\Hankyung_Data.json'

# Main

## https://kr.investing.com/news/cryptocurrency-news

### 크롤링

In [24]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
start = 1
get_page_cnt = 2000
end = start + get_page_cnt
p0 = re.compile(r'\s+\r\n\s+')
p1 = re.compile(r'(읽기|provided|[a-z0-9]@[a-z0-9]|무단전재|재배포|쿠폰코드)', flags=re.IGNORECASE)
p2 = re.compile(r'([가-힣])\.(\w)')
p3 = re.compile(r'(\xa0)+')
# 일부 파라미터들을 고정한 investing 함수 생성
fixed_investing = partial(investing,
                            headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry,
                            p0=p0, p1=p1, p2=p2, p3=p3)
results = []

for i in tqdm(range(start, end)):
    try:
        html = requests.get(web_page, headers=headers, allow_redirects=allow_redirects, timeout=timeout).text
        soup = BeautifulSoup(html, 'html.parser')
        url_tag_list = soup.find_all('article', {"data-test": "article-item"})
        url_list = [url["href"] for url_tag in url_tag_list if (url := url_tag.find('a') is not None)]
    except Exception as e:
        print()
        print(f'{i}번 페이지의 HTML 문서 정보를 불러오는데 실패했습니다.')
        print(traceback.format_exc())
        web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i + 1}'
        continue

    try:
        executor = futures.ThreadPoolExecutor(max_workers=workers)
        data = executor.map(fixed_investing, url_list)

        for idx, d in enumerate(data, start=1):
            if d is not None:
                results.append(d)
            else:
                print()
                print(f'-{i}번 페이지-')
                print(f'{i}번 페이지의 {idx}번째 데이터를 가져오는 것에 실패했습니다.')
                print(f'실패한 뉴스 데이터의 URL : {url_tag_list[idx - 1].find("a")["href"]}')

    except Exception as e:
        print()
        print(f'-{i}번 페이지-')
        print(f'{d}를 가져오는 것에 실패했습니다.')
        print(traceback.format_exc())

    time.sleep(random.uniform(0.35, 1.25))
    web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i + 1}'

 35%|███▍      | 697/2000 [1:13:05<2:15:08,  6.22s/it]


-698번 페이지-
{'news_title': '[도기자의 한 주 정리] 비트코인 상승 랠리 놓쳤다면 주목해야 할 주식 12개', 'news_first_upload_time': '2021-10-15 PM 05:30', 'news_last_upload_time': None, 'author': None, 'news_content': '비트코인\n(BTC) 상승 랠리가 지속되면서 암호화폐 시장에 대한 투자자 관심도 높아지고 있습니다. 그런데 만약 이번 기회를 놓쳤다면 어떻게 해야 할까요? 뱅크오브아메리카는 주식에서 기회를 노리라고 조언했습니다. 뱅크오브아메리카가 지목한 12개 종목에는 월트디즈니, 워너뮤직 등 큰 관련이 없어 보이는 종목도 있는데요.\n자세한 내용 함께 살펴보도록 하겠습니다.\n한 주 간 이슈를 콕 집어 정리해 드리는 도기자의 한 주 정리입니다.\n?지난 12일(현지시간) CNBC는 뱅크오브아메리카 (NYSE:\nBAC\n) 애널리스트가 12개 종목을 추천했다고 보도했습니다. 뱅크오브아메리카는 “암호화폐는 무시하기에는 자산 규모가 너무 크다”며 “흐름을 타고 싶다면 주목해야 할 종목이 있다”고 분석했습니다.\n━\n암호화폐 사업에 적극적인 금융사 3곳\n암호화폐 관련 금융주로는 페이팔(PYPL), JP모건체이스(JPM), 모건스탠리(MS)를 추천했습니다. 페이팔은 비트코인을 직접 매입하기도 했고, 관련 결제 시스템 도입에도 속도를 내고 있습니다. 모건스탠리도 지난 3월 일반 투자자 대상 비트코인 펀드 3종을 출시했습니다.\n흥미로운 종목은 JP모건체이스입니다. 제이미 다이먼 JP모건체이스 회장은 암호화폐 비판론자로 알려져 있습니다. 비트코인에 자산가치가 없다고 비판했습니다. 수장이 나서서 비트코인을 비판하고 있지만 이는 개인 의견일 뿐 회사 차원에서 관련 사업에 발빠르게 진출했습니다. JP모건은 지난 2019년 미국계 은행으로는 최초로 암호화폐를 지원했습니다. 지난 4월에는 부유층 고객을 상대로 비트코인 펀드에 대한 투자 제안도 준비하고 있는 것으

100%|██████████| 2000/2000 [3:28:17<00:00,  6.25s/it]  


### JSON으로 저장

In [25]:
with open(investing_path, 'w') as f:
    json.dump(results, f, ensure_ascii=False)

## https://www.hankyung.com/koreamarket/news/crypto

### 크롤링

In [28]:
start = 1
get_page_cnt = 200
end = start + get_page_cnt
p0 = re.compile(r'([a-z0-9]@[a-z0-9]|사진=|\b기자\b|영상촬영\s*:|한국경제TV [가-힣]{2,4}입니다)', flags=re.IGNORECASE)

# 일부 파라미터들을 고정한 hankyung 함수 생성
fixed_hankyung = partial(hankyung,
                            headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry,
                            p0=p0)
results = []

for i in tqdm(range(start, end)):
    web_page = f'https://www.hankyung.com/koreamarket/news/crypto?page={i}'
    try:
        html = requests.get(web_page, headers=headers, allow_redirects=allow_redirects, timeout=timeout).text
        soup = BeautifulSoup(html, 'html.parser')
        url_tag_list = soup.find_all('h2', {"class": "news-tit"})
        url_list = [url["href"] for url_tag in url_tag_list if (url := url_tag.find('a') is not None)]
    except Exception as e:
        print()
        print(f'{i}번 페이지의 HTML 문서 정보를 불러오는데 실패했습니다.')
        print(traceback.format_exc())
        continue

    try:
        executor = futures.ThreadPoolExecutor(max_workers=workers)
        data = executor.map(fixed_hankyung, url_list)

        for idx, d in enumerate(data, start=1):
            if d is not None:
                results.append(d)
            else:
                print()
                print(f'-{i}번 페이지-')
                print(f'{i}번 페이지의 {idx}번째 데이터를 가져오는 것에 실패했습니다.')
                print(f'실패한 뉴스 데이터의 URL : {url_tag_list[idx - 1].find("a")["href"]}')

    except Exception as e:
        print()
        print(f'-{i}번 페이지-')
        print(f'{d}를 가져오는 것에 실패했습니다.')
        print(traceback.format_exc())

    time.sleep(random.uniform(0.35, 1.25))

100%|██████████| 200/200 [11:42<00:00,  3.51s/it]


### JSON으로 저장

In [29]:
with open(hankyung_path, 'w') as f:
    json.dump(results, f, ensure_ascii=False)

In [41]:
url = 'https://kr.investing.com/news/cryptocurrency-news/article-1302958'
html = requests.get(url, headers=headers, allow_redirects=allow_redirects, timeout=timeout).text

In [42]:
soup = BeautifulSoup(html, 'html.parser')

In [43]:
article = soup.find('div', id='article')
content = article.find_all(re.compile(r'^(p|)$'))
content = article.find_all(lambda tag: tag.name in ('p', 'span', 'h2'))

In [44]:
'\n\n'.join(t.text for t in content).strip(' \t\n\r\f\v')

'가상자산법의 이해를 돕기 위해 영상 콘텐츠를 제작한 코빗은 오는 13일부터 2주 동안 소셜네트워크(SNS) 퀴즈 이벤트를 진행한다. 코빗 공식 유튜브 채널에서 ‘가상자산 이용자라면 알아야 할 Q&A’를 시청하고 퀴즈를 맞힌 이용자에게 추첨을 통해 상품을 제공한다.\n\n코빗은 영상을 통해 이용자 자산 보호와 거래 시 주의사항을 알리는 것에 중점을 뒀다. 또 이번 이벤트를 통해 이용자가 안전하게 가상자산 시장에 접근하도록 지원할 방침이다. 이벤트에 대한 자세한 내용은 코빗 공식 홈페이지와 애플리케이션(앱)에서 확인할 수 있다. 오세진 코빗 대표는 “가상자산법을 준수하고 내부통제를 강화해 바람직한 가상자산 거래 문화를 조성하겠다”고 말했다.\n\n가상자산법은 이용자 자산 보호를 강화하고 불공정 거래를 규제하기 위해 마련된 법안으로, 주요 내용은 △이용자 예치금 보호 △콜드월렛 보관 의무 △불공정 거래 감시 등이다.\n\n디센터에서 읽기'

In [49]:
t = article.select('#div > span')
t

[]